# Introduction

`metadsl` inserts a layer between calling a function and computing its result, so that we can build up a bunch of calls, transform them, and then execute them all at once.

## Instances and Calls

The two building blocks we start with are, `Call`s and `Instance`s:

In [1]:
import dataclasses
import metadsl

@dataclasses.dataclass
class MyObject(metadsl.Instance):
    @metadsl.call(lambda self: MyObject)
    def do_things(self) -> "MyObject":
        ...

    @metadsl.call(lambda self, other: MyObject)
    def __add__(self, other: "MyObject") -> "MyObject":
        ...

        
@metadsl.call(lambda x: MyObject)
def create_object(x: int) -> MyObject:
    ...

In [2]:
o = create_object(123)
o

MyObject(__call__=Call(create_object, (123,)))

In [3]:
o.do_things() + o

MyObject(__call__=Call(__add__, (MyObject(__call__=Call(do_things, (MyObject(__call__=Call(create_object, (123,))),))), MyObject(__call__=Call(create_object, (123,))))))

It is useful to keep in mind the strict typing constraints here, not all of which can be faithfully checked by MyPy:

1. The arguments in a `Call` should fit the signature of the function in the call.
2. The call within the instance should have represent a function whose return type is the type of the instance.
3. The `type_fn` is the first argument passed into the `call` decorator. It should take in the same signature as the function itself, but return a function that maps from a `Call` to the return value.

Let's check the first two of these for `o`. We see that the call's function is `create_object`, which takes in an `int` and returns a `MyObject`. It's argument is indeed an `int`, so the first is true. And the instances holding it is of type `MyObject`, which is its return type.

For the third, we see that for both calls, it is simply creating a new `MyObject` with the call.

## Expressions and Recursive Calls

You might notice that this representation is a bit verbose. It actually contains unneccesary information, which is the type that each function returns. We don't need to store this,
because we can always recreate it given the type functions we have specfifed.

We can translate these instances into `RecursiveCall`s, which have the same structure as `Call` object but different typing constraints:

In [4]:
r = metadsl.to_expression(o.do_things() + o)
r

RecursiveCall(__add__, (RecursiveCall(do_things, (RecursiveCall(create_object, (123,)),)), RecursiveCall(create_object, (123,))))

In [5]:
print(r)

__add__(do_things(create_object(123)), create_object(123))


The relationships here are:
1. The result of `to_expression` is a `RecursiveCall` if the argument is a `Instance` and the original object otherwise. 
2. The args in a `RecursiveCall` should either be the right type for the function, if they are not an `Instance`, or a `RecursiveCall` that will return that `Instance` subtype, if they are.

We can always get back to original nested instance version of the call:

In [6]:
metadsl.from_expression(r)

MyObject(__call__=Call(__add__, (MyObject(__call__=Call(do_things, (MyObject(__call__=Call(create_object, (123,))),))), MyObject(__call__=Call(create_object, (123,))))))

This will recursively transform expression recursively. This leads us to another relationship:

1. `from_expression(to_expression(x)) == x` for all `x`, if it is in `Instance` or any other Python object.

Why do we have these two forms? Well we need the `Instance` form, to provide a nice typed Python API. You should always be dealing in that form when you are calling the Python functions, so that the typing is enforced by MyPy. However, it's a bit more complicated to modify the graph in this form. So whenever we are traversing the graph in some way, we first convert it to expressions, so that we don't have to deal with the intermingled types.

## Replacements

Often, it's helpful to think about replacements we do on the graph. By writing logic like this, we can then apply it to any nodes on the graph any number of times.

We can combine a bunch of replacements and have them execute repeatedly on all nodes of the graph, until no more apply:

In [8]:
replacements = metadsl.Replacements()

@replacements.register(None, None)
def _add(x: int, y: int):
    return create_object(x) + create_object(y), lambda: create_object(x + y)

@replacements.register(None)
def _do_things(x: int):
    return create_object(x).do_things(), lambda: create_object(x * 2)

The requirements for these replacements is that they take in some arguments, which can match any expression in the graph.
Their first return value, build up template expression based on the inputs, that shows what it should match again. The second
is a thunk that returns the resulting replacement. Note that both should have the same type, because all replacements should be equivalent.

We can call these on an instance and it will return a replaced version of it:

In [9]:
print(o)
print(replacements(o + o))

MyObject(__call__=Call(create_object, (123,)))
MyObject(__call__=Call(create_object, (246,)))


Just like we have a function that creates a `MyObject` from an `int`, we can have a similar one that does the opposite:

In [11]:
@metadsl.call(lambda o: replacements)
def unwrap_object(o: MyObject) -> int:
    ...
    
@replacements.register_pure(None)
def _unwrap_object(i: int):
    return unwrap_object(create_object(i)), i

unwrap_object(o + o)

246

This is nice, because now we can write our unboxing as a replacement, which means it's nice and type safe.

You notice that here we are returning `replacements` from the `type_fn`. This means that `replacements` will be called with the created `Call(unwrap_object, (o))` object, which will in turn call all the replacements on it, 
including that which we defined below for the unwrapping. So this *should* return an `int`, so it does violate the typing constraints we first wrote out about `call`s.